# Logistic Regression - Predicting shipwreck survival

In this week's assignment you will implement a classification algorithm named Logistic Regression. This sounds very counter-intuitive, as regression is something entirely different than classification, but the Logistic Regression model is actually estimating the probability (which is a continuous value) that data can be assigned to a specific class out of a set of classes. We will focus on so-called binary classification problems, wherein there are two possible classes. Some of the examples of binary classification problems that Logistic Regression is able to solve are: 

- Email; spam or not spam 
- Online transactions; fraud or not fraud
- Tumor; malignant or benign 

In this notebook, we will be using the classic Titanic dataset. This data consists of demographic and traveling information for 891 of the Titanic's passengers, and the goal is to predict which of these passengers survived. 

Here is a summary of the data set's attributes:

- *PassengerId*: passenger ID assigned in this dataset
- *Survived*: A Boolean indicating whether the passenger survived or not (0 = No; 1 = Yes); this is our target
- *Pclass*: Passenger class (1 = 1st; 2 = 2nd; 3 = 3rd)
- *Name*: field containing the name and title of the passenger
- *Sex*: male/female
- *Age*: age of the passenger in years
- *SibSp*: number of siblings/spouses aboard
- *Parch*: number of parents/children aboard
- *Ticket*: ticket number
- *Fare*: passenger fare in British Pounds
- *Cabin*: location of the cabin, consisting of a letter indicating the deck, and a cabin number.
- *Embarked*: port of embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)

As you can probably see, the dataset contains a mix of textual, continuous, categorical, and boolean variables. Before we can get into implementing and applying Logistic Regression, we will have to clean up the data. We will lead you through this process using functions a data-scientist might use, providing you with the tools that will help you prepare your own data in the future.

In the code below, we have loaded the data and show its first few rows.

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('titanic.csv')

data.head()

### A quick view

From the first five rows displayed in the table above, you might be able to see that there are multiple types of variables: we have integers, strings, floats, and a few *NaN*'s, which are missing values. Lets take a quick look to see what the types of data that we are dealing with actually are. A tool a data-scientist might use here is Panda's [`df.info()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html), which is a method that can be applied to any `DataFrame` that shows information about a that frame, including the index, datatype for each column, non-null values, and memory usage.

In [ ]:
data.info()

As you can see we have 891 entries, numbered 0 to 890, with 12 different variables each. Of these 12 variables, *Survived*, will be our target variable. Out of 12 columns, 2 are floats, 5 are integers, and 5 are "objects" which is the generic type, but in this case these are strings. Out of all of the columns, there are 3 that have some missing values. We will need to fix these missing values later.

Let's take a better look at what the numeric columns, i.e. those that contain integers and floats, look like. For this, we will use Panda's [`df.describe`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html?highlight=describe#pandas.DataFrame.describe), which is a method that can be applied to any `DataFrame` to show its descriptive statistics, which summarize the central tendency, dispersion and shape of a dataset’s distribution, excluding NaN values.

In [ ]:
display(data.describe())

The result is a table showing some statistics for each of the columns. The `count` is the number of values that is not NaN in that specific column. We can also see mean, standard deviation, minimum, maximum, and percentiles for each of the columns. Percentiles are covered in *SOWISO* chapter 2 (Describing Data), the $50^{th}$ percentile corresponds to the median value. Let's see what we can conclude from this table and the previous one:

- *PassengerId* is a column that runs from 1 to 891, indicating the index of a passenger in this dataset. This variable has no descriptive value for the passenger, but it could still cause our machine learning model to find some correlations that are really just noise, so we should delete this whole column before using the data for training.
- The mean of *Survived* indicates that out of the 891 people in our dataset, only ~38% of people survived. This can be deduced from the mean of the column, because the value 1 corresponds to survived and 0 corresponds to did not survive. As an example, when 90% of these values would be 1, we would end up with a mean of 0.9, as the other values are all zeros.
- There are more people in *Passenger Class* 3 than in class 1 and 2 combined. This can be concluded from the fact that from the 50th percentile to the maximum value, every passenger is in third class.
- There is at least one baby in our dataset, with an *Age* of 0.42, or 5 months. The oldest person is 80 years old. We can see this from the min and max values of the *Age* column.
- There are 277 people of which we do not know the *Age*. We have 891 data entries, but the `count` for age is 714, so 891 - 714 = 277.
- The majority of people has no *Siblings/Spouses* aboard. This can be concluded from the percentiles.
- The majority of people has no *Parents/Children* aboard. This can be concluded from the percentiles.
- There is a big difference in scale in data. Fare can be up to 512 pounds while the number of *Parents/Children* on board is only up to 6.  We can also see this in the standard deviations, where the difference is also very large. In his videos, Andrew Ng talks about feature scaling, which is something we should apply here. We will discuss this later.

Often, these tables can tell you a lot about the data you are working with, and might provide you with intuitions on what would work best for this dataset. We will spend some more time on analysing the data at the end of this notebook.

### Cleaning the data

Let's create a copy of the dataset that does not contain data that we won't need for Logistic Regression. In this case, we will delete the rows containing the *PassengerId*, *Name*, *Cabin Number*, or *Ticket number* of each passenger. In a more advanced data processing pipeline we might still include some of this data, but that would include data mining techniques that are beyond the scope of this notebook. The cabin number probably isn't directly useful, but could perhapse be converted to information on where the passenger was on the ship, which in turn might give information on if a passenger would have been able to reach a lifeboat on time.

Create a new DataFrame called `clean_data` that is a direct copy of our original `data`, but does not include the colums *PassengerId*, *Cabin*, *Name* and *Ticket*. Take a look at Panda's [`df.drop`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) which can be used to delete columns from a given `DataFrame`, when setting the `axis` parameter correctly. The method returns a `Dataframe` without the rows/columns that were deleted.

In [ ]:
### YOUR SOLUTION HERE


In our analysis of the data, we concluded that there were 277 people of which we do not know the age. One could say that the absence of information is also information, but in our case there is no way that our model can calculate a prediction when one of the input variables is absent. When you encounter missing values, you have multiple options for dealing with these missing values:

* You can delete each row that contains a missing values
* You can delete the whole column from the DataFrame
* You can replace the missing values.

For this data set, deleting the rows would not really be an option, as we already do not have a lot of data, and we need as many training samples much as possible to create an accurate model. Deleting the column would also not be an option, as there are many cases were we *do* have the age and the age of a person will probably be a good indicator for whether a person survived or not. The third option, replacing the missing values, remains, but we have to find a method that will not skew our results. 

There are multiple ways of replacing missing values, but it is important to find a method that does not affect the outcome of our model too much. Replacing the values randomly will probably create a lot of outliers and will probably result in our model not being able to learn what correlations there are within the data, so we will use a method that replaces missing values with values that are as generic as possible. The way to do this is to replace all missing values with a value that is inferred from the data that is available, where the easiest option is to just use the *mean* of all the non-missing values as the replacement value.

In the cell below, use Panda's [`df.fillna`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html?highlight=fillna#pandas.DataFrame.fillna) to replace every missing value in the age column of `clean_data` with the average age based on all the non-missing values.

*Hint*: `df.fillna` accepts a value that will be used to replace all missing values. It returns a new DataFrame or Series, depending on you call the function on. Use `mean()` on the *Age* column to get the average age, and then select the column *Age* and apply `fillna` with this mean value. Store the result in the age column to overwrite it.

In [ ]:
### YOUR SOLUTION HERE


display(clean_data.describe())

As you can see, all missing values in the column *Age* have now been replaced with the average. This means that the average of the column has not changed. However, this solution is still not ideal. We have changed the standard deviation and the percentiles, which indicates that we have changed the entire distribution of the data. As there is no way to recover the real data, for now we will settle for this solution.

Finally, we will have to transform the columns *Sex* and *Embarked* from categorical data into a type of data that the Logistic Regression can handle. The column *Sex* has two categories, male or female, while the port where passengers *Embarked* has three categories; C = Cherbourg, Q = Queenstown, or S = Southampton. In the previous module, you learned about one-hot encoding, which is exactly what we will use here too. Panda's has a method named [`get_dummies`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) that transforms data from categorical to a one-hot encoded set of columns.

Note that, for now, we will choose to interpret the *Passenger Class* as a numeric variable, as we won't one-hot encode it. However, you could make the argument that it is actually a categorical variable, as it only really has three values. In this case, both interpretations of the variable are valid, as it is a categorical variable with ordinal properties; second class is actually something "between" first and third class. Using the variable as if it is numerical however *does* imply that the difference between first and second class is the same as the difference between second and third class. Using the variable as if it is categorical implies that there is no order, and that the three different classes have no connection other than that you can only be part of one class.

In [ ]:
# We can drop one of the sexes, as being one of the sexes infers that the passenger can no longer be the other
sex = pd.get_dummies(clean_data['Sex'], drop_first=True)

# This is not the case for port of embarkment, as there are three possibilities here
embark = pd.get_dummies(clean_data['Embarked'])

# We no longer need these columns, as we will replace them soon
clean_data = clean_data.drop(['Sex','Embarked'],axis=1)

# Create the new dataframe with the one-hot columns we encoded from the categories in Sex and Embarked
clean_data = pd.concat([clean_data, sex, embark],axis=1)

clean_data.info()
clean_data.head()

### Final preparations

Now that all data is numerical, we can continue to the next step. We have already observed that there is a big difference in scale in data. When using feature scaling you make sure that the different features in your dataset take on similar ranges of values. One of the ideas behind this is that after you apply feature scaling, gradient descent can converge more quickly.

Scipy's `stat` module has some interesting methods to do this, including the `zscore` method which you might recognise from the theory video:

$$ \tilde{\mathbf{x}}_j = \frac{\mathbf{x}_j - \mu_j}{\sigma_j} $$

Here $\mathbf{x}_j$ is a vector of x-values for the $j^{th}$ column, $\mu_j$, $\sigma_j$ are the (scalar) mean and standard deviation for that same column and $\tilde{\mathbf{x}}_j$ is the scaled version of the feature vector. The basic principle here is that it transforms every value in the column $j$ to a value that indicates how many standard deviations the value originally was away from the mean of the data *in that column*. For example: a $Z$-score of $-2$ would indicate that the value was 2 standard deviations below the average, while a $Z$-score of $0$ would indicate that the value was actually equal to the mean of the column.

In the code below, we transform each of the non-categorical datacolumns using the `zscore` method, by applying it to each of the elements within. The function `zscore` conveniently takes care of computing all means and standard deviations for each column separately, and returns the scaled data.

Note that we do not want to apply this function to our categorical one-hot encoded data, as those are already in within a nice $[0,1]$ range and scaling these binary values by their mean and standard deviation wouldn't really make sense.


In [ ]:
from scipy.stats import zscore

numerical = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
clean_data[numerical] = clean_data[numerical].apply(zscore)

display(clean_data.describe())

The columns that we have adjusted with our `zscore` function now all have a mean that is very close to zero, and a standard deviation that is very close to one. So, the resulting data is now sufficiently of equal scale, and mean-centered.

The final thing that we have to do to prepare the data, is to separate the input variables from the target variable, and split the dataset into a training and test set. For this we'll use the `sklearn` function `train_test_split`

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into target "y" and input "X"
y = clean_data['Survived']
X = clean_data.drop('Survived', axis=1)

#Split the data into 70% training and 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

### The Logistic function

For Logistic Regression, we require a function that generates probabilities; a function that gives outputs between 0 and 1 for **all** inputs. There are many functions that meet this description, but the one that is used in logistic regression is the logistic function:

$$ g(z) = \frac{1}{1+e^{-z}} $$

Implement the function `logistic_func` that can either take a single value `z`, or an array of values `z`, and compute the *Logistic* function for each.

*Hint:* Numpy built-in functions and basic arithmetic operations work on both Numpy arrays and single values. When the function detects that its input is a Numpy array, the operation it applies will be applied to each element in the array seperately, the result being a Numpy array of the same shape as the input. This is called *element-wise application*, and many mathematical operations have Numpy version that support element-wise application. Write your `logistic_func` in such a way that it can compute $g$ values for scalars or for entire vectors.

Then, write some code that can plot the results for the logistic function between $-5$ and $5$. Use Numpy's [`np.linspace`](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html?highlight=linspace#numpy.linspace) to generate a range of $z$ values and use your `logistic_func` to calculate the corresponding $g$ values. Make sure the resulting plot looks correct, before moving on to the next step.

In [ ]:
def logistic_func(z):
    ### YOUR SOLUTION HERE
    

### YOUR SOLUTION HERE


### Hypothesis function

Next, we can start working on the hypothesis function, which will generate a hypothesis vector of class probabilities, given some matrix of inputs $X$ and a vector of weights $\theta$. For logistic regression, we use the logistic function $g$ to transform the hypothesis we used in multivariate linear regression to a hypothesis function that only generates probabilities (i.e. results between 0 and 1):

$$ h_\theta(x^1) = g(\theta^Tx^1)$$
$$ h_\theta(x^2) = g(\theta^Tx^2)$$
$$ h_\theta(x^3) = g(\theta^Tx^3)$$
$$\dots$$
$$ h_\theta(x^n) = g(\theta^Tx^n)$$

The notation is a little confusing here, as $x^i$ is vector and the default shape for vectors is a *column* vector, even if $x^i$ in the complete data matrix $X$ is a *row* vector. This will actually get somewhat easier when we write out the full vectorized version using the matrix $X$, as the rows and columns get their "expected" shape. 

Here the function $g$ is the Logistic function defined above. As you've just written `logistic_func` specifically to also work on vectors, if we construct a vector of $z$ values for the whole model at once, then we could also write the entire hypothesis function in vectorized form:

$$ \left[\begin{array}{cccc}
x_0^1 & x_1^1 & x_2^1 & \cdots & x_n^1 \\ 
x_0^2 & x_1^2 & x_2^2 & \cdots & x_n^2 \\ 
x_0^3 & x_1^3 & x_2^3 & \cdots & x_n^3 \\ 
\vdots & \vdots & \vdots & \ddots & \vdots \\
x_0^m & x_1^m & x_2^m & \cdots & x_n^m \\ 
\end{array} \right]
\left[\begin{array}{c} \theta_0 \\ \theta_1 \\
\theta_2 \\ \vdots \\ \theta_n \end{array} \right]
= \left[\begin{array}{cccc}
z^1\\
z^2\\
z^3\\
\dots \\
z^m \\
\end{array} \right]
$$

Next, we can apply the Logistic function to each of the elements of this $\mathbf{z}$ vector:

$$
\left[\begin{array}{cccc}
g(z^1)\\
g(z^2)\\
g(z^3)\\
\dots \\
g(z^m) \\
\end{array} \right]
=
\left[\begin{array}{c} h_{\theta}(x^1) \\ h_{\theta}(x^2) \\
h_{\theta}(x^3) \\ \vdots \\ h_{\theta}(x^m) \end{array} \right]
$$

Large parts of these equations should look familiar, as they're really just *Multivariate Linear Regression* with an extra *Logistic function* applied. This means you can (and should) take inspiration from your multivariate linear solutions, although some of the details will be a little different here.

First, write a function `add_x0`, which takes a matrix `X` and prepends a column vector of constant $1$ values to this matrix, returning the new matrix with this constant factor added for each row. Next, write the function `logistic_model` to compute the hypothesis vector of the model for some matrix `X` and parameter vector `theta`. You should assume that `X` here already has had $x_0$ added the matrix. Note, your solution for `logistic_model` should once again not use any loops.

In [ ]:
def add_x0(X):
    ### YOUR SOLUTION HERE
    

def logistic_model(X, theta):
    ### YOUR SOLUTION HERE    
    

X_train_x0 = add_x0(X_train)
X_test_x0 = add_x0(X_test)

### Cost function

Now that we can make predictions using an input matrix $X$, and our parameter weights $\theta$, we can evaluate the cost of our model. To be able to apply gradient descent, we will need a cost function that is convex when we use our logistic model to make predictions. This cost function for logistic regression is explained in detail in the theory videos, and is defined as:

$$J(\theta) = - \frac{1}{m} \sum_{i=1}^M y^i log(h_\theta(X^i)) + (1 - y^i) log(1 - h_\theta(X^i))$$

Implement the function `logistic_cost`. Use your `logistic_model` function to calculate the hypothesis vector $h_\theta(X)$. Just as with the cost function for *Multivariate Linear Regression*, it is actually possible to write this whole function using only linear algebra, and some element-wise function applications, although this once is definitely a little harder.

*Hints:* Most mathematical functions have a *Numpy* version that supports element-wise application, which will really help here. If you're stuck on how to write all of this as matrix multiplications, you can also use the element-wise version of multiplication [np.multiply](https://numpy.org/doc/stable/reference/generated/numpy.multiply.html). Addition and subtraction are already element-wise by default, so all that would remain is summing the resulting vector.

In [ ]:
def logistic_cost(theta, X, y):
    ### YOUR SOLUTION HERE
    


## Obtaining the Gradient terms for Logistic Regression

Manually taking the whole partial derivative of this cost function can be a little time consuming, but in order to still practice this skill, we'll just do a part of this derivative instead. For this part of the notebook, you should consult the `logistic_derivative.pdf` supplement, included as a part of the assignment. Read sections 1, 2 and 3 in the supplement, and then complete the steps of the partial derivative below, labeling each of the steps with their respective rules (as with the linear regression assignment).

$$\frac{\partial E_{\mathbf{\theta}}^i}{\partial h_{\mathbf{\theta}}^i} =
\frac{\partial}{\partial h_{\mathbf{\theta}}^i} -y^i log(h_{\mathbf{\theta}}^i) -
(1 - y^i) log(1 - h_{\mathbf{\theta}}^i)$$

**TODO:** *Your steps, each labeled with what rules you've applied, should go here*.

$$\frac{\partial E_{\mathbf{\theta}}^i}{\partial h_{\mathbf{\theta}}^i} =
\frac{-y^i}{h_{\mathbf{\theta}}^i} +
\frac{1 - y^i}{1 - h_{\mathbf{\theta}}^i}$$

Section 4 of the supplement is optional, but definitely read sections 5 and 6 before moving on to the next step. 

### Gradient Vector and Gradient Descent

In the end, the partial derivatives for each of the $\theta$ parameters look the exactly same as for polynomial regression. While the applied hypothesis function $h$ changes, the rest of the derivative does not, even though the cost function is completely different. As a result, the equations for the $n+1$ partial derivatives for each of the $n+1$ parameters, in the notation of the theory videos, then just become:

$$\frac{\partial J}{\partial \theta_0} = \frac{1}{m} \sum_{i=1}^m (h_{\theta}(x^i) - y^i)x^i_0$$
$$\frac{\partial J}{\partial \theta_1} = \frac{1}{m} \sum_{i=1}^m (h_{\theta}(x^i) - y^i)x^i_1$$
$$\frac{\partial J}{\partial \theta_2} = \frac{1}{m} \sum_{i=1}^m (h_{\theta}(x^i) - y^i)x^i_2$$
$$\dots$$
$$\frac{\partial J}{\partial \theta_n} = \frac{1}{m} \sum_{i=1}^m (h_{\theta}(x^i) - y^i)x^i_n$$

Write the functions `gradient_vector` and `gradient_descent` for this *Logistic Regression* model. You should be able to reuse most of your code from *Multivariate Linear Regression* here.

In [ ]:
def gradient_vector(theta, X, y):
    ### YOUR SOLUTION HERE
    
    
def gradient_descent(X, y, theta, alpha, thres=10**-6):
    ### YOUR SOLUTION HERE
    

# Initialize theta to a zero vector of the correct shape
theta = np.zeros(X_train_x0.shape[1])

# Find the theta vector that minimizes the cost function
theta_hat = gradient_descent(X_train_x0, y_train, theta, 10**-5)

print(theta_hat)

### Predictions

We now have a learned vector of weights $\theta$ that we can use to make predictions for our test samples `X_test`. We can now actually use our hyptohesis function to calculate the probability that each of the samples in `X_test` is a survivor or not:

$$P(y^i=1|x^i,\theta) = h_{\theta}(x^i)$$

Finally, these estimates of the probability that $y=1$ still need to be transformed into boolean predictions, which will be the actual classifications. In the end, this regression model should still predict that either the person has survived $y=1$, or that the person has not survived $y=0$.

Write the function `predict`, that takes a matrix of input values `X` and a vector of weights `theta`, and transforms them into a vector of boolean predictions. Use `logistic_model` to generate predictions, and then use a decision boundary of $h_\theta(x) \geq 0.5$ to determine when you should classify each sample as $y=1$.

In [ ]:
def predict(X, theta):
    ### YOUR SOLUTION HERE
    

predictions = predict(X_test_x0, theta_hat)

### Determining accuracy

Next, we would like to see how *accurate* our model is, now that we have trained it and can generated prediction. Implement the function `calc_accuracy` that accepts a vector of `predictions` and a vector of *ground-truth* values `y`. The function should return the ratio of correct predictions (predictions where the value in `predictions` is equal to the value in `y`). The ideal situation where every classification is correct, should result is an accuracy of $1$.

In [ ]:
def calc_accuracy(predictions, y):
    ### YOUR SOLUTION HERE
    

print(calc_accuracy(predictions, y_test))

### Confusion matrix

The accuracy we have just determined is of course one of the primary goals of building a model; we generally want a Machine Learning model to be as accurate as possible. However, it does not give much insight into in what way our model was accurate. As an example, let's say that we want to diagnose whether a patient has a virus. Of course we want our model to diagnose as accurately as possible, but there are two types of error the model could make:

1. A patient *has* the virus, but was diagnosed as being negative
2. A patient *does not have* the virus, but was diagnosed as being positive

One of these types of mistakes is potentially *much* more costly than the other, as falsely diagnosing a patient as negative could become very harmful for the patient and their environment. Although an extreme (and currently very relevant) example, it is quite often the case that in classification that not every error has the same associated *cost*.

To give better insight into the performance of a classification algorithm, typically, a confusion matrix is made. The confusion matrix is a table layout that allows quick identification of the performance of a classification algorithm. Each *row* in the matrix represents the data that got predicted as a specific class, while each *column* represents the data that is actually in a class:

<table>
    <thead>
        <tr> <th colspan=2></th><th colspan=2 style="border: 1px solid black;"> Actual class </th> </tr>
        <tr>
            <th colspan=2></th>
            <th style="border: 1px solid black;">P</th>
            <th style="border: 1px solid black;">N</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td rowspan=2 style="border: 1px solid black;"><b> Predicted class </b></td>
            <td style="border: 1px solid black;"><b> P </b></td>
            <td style="border: 1px solid black;"><b> TP </b></td>
            <td style="border: 1px solid black;"> FP </td>
        </tr>
        <tr>
            <td style="border: 1px solid black;"><b> N </b></td>
            <td style="border: 1px solid black;">FN</td>
            <td style="border: 1px solid black;"><b> TN </b></td>
        </tr>
    </tbody>
</table>

Where:
- P = Positive
- N = Negative
- TP = True Positive, denoting every value that was correctly predicted as positive
- FP = False Positive, denoting every value that was predicted as positive but was actually negative
- TN = True Negative, denoting every value that was correctly predicted as negative
- FN = False Negative, denoting every value that was predicted as negative but was actually positive

For example, using the example of diagnosis of patients, let's say that 100 people take a test, and of these people, 80 actually have the virus. We have two different testing kits that are used to diagnose the patients, and these are the resulting confusion matrices:


##### Testing kit 1
<table>
    <thead>
        <tr> <th colspan=2></th><th colspan=2 style="border: 1px solid black;"> Actual class </th> </tr>
        <tr>
            <th colspan=2></th>
            <th style="border: 1px solid black;">P</th>
            <th style="border: 1px solid black;">N</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td rowspan=2 style="border: 1px solid black;"><b> Predicted class </b></td>
            <td style="border: 1px solid black;"><b> P </b></td>
            <td style="border: 1px solid black;"><b> 56 </b></td>
            <td style="border: 1px solid black;"> 1 </td>
        </tr>
        <tr>
            <td style="border: 1px solid black;"><b> N </b></td>
            <td style="border: 1px solid black;"> 24 </td>
            <td style="border: 1px solid black;"><b> 19 </b></td>
        </tr>
    </tbody>
</table>

##### Testing kit 2
<table>
    <thead>
        <tr> <th colspan=2></th><th colspan=2 style="border: 1px solid black;"> Actual class </th> </tr>
        <tr>
            <th colspan=2></th>
            <th style="border: 1px solid black;">P</th>
            <th style="border: 1px solid black;">N</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td rowspan=2 style="border: 1px solid black;"><b> Predicted class </b></td>
            <td style="border: 1px solid black;"><b> P </b></td>
            <td style="border: 1px solid black;"><b> 62 </b></td>
            <td style="border: 1px solid black;"> 10 </td>
        </tr>
        <tr>
            <td style="border: 1px solid black;"><b> N </b></td>
            <td style="border: 1px solid black;"> 18 </td>
            <td style="border: 1px solid black;"><b> 10 </b></td>
        </tr>
    </tbody>
</table>

Now, we can learn a lot from these confusion matrices. First, the calculated accuracy of our first virus diagnosis testing kit is 77%, as 56 people are correctly diagnosed as positive, and 19 people are correctly diagnosed as negative.  The calculated accuracy for the second testing kit is 72%. However, using the first testing kit 24 people have been incorrectly diagnosed as negative, while for the second this was 18. Now, arguably, we would prefer to use testing kit 2, as even though the total accuracy of this testing kit is lower, this kit produces fewer (harmful) false negatives. 

Implement the function `confusion_matrix` that, given `predictions` and truth values `y`, creates a $2 \times 2$ Numpy matrix that contains the number of TP, FP, FN, and TN for these predictions.

In [ ]:
def confusion_matrix(predictions, y):
    ### YOUR SOLUTION HERE
    

print(confusion_matrix(predictions, y_test))

#### Which type of error is larger for the Logistic Regression model on this Titanic data set; false positives or false negatives?

*Your answer goes here.*

### Correlations and understanding weights

To better understand what relations our model has found in the data we will have to take a look at the weights that our model has learned. The relative size of each weight is a direct indication of how important the corresponding feature is in defining the relationship between the input features and the target output. We can even argue why this is the case, as the hypothesis of our model is entirely dependant on the linear combination of the input values and our weights:

$$ h_{\theta}(x^i) = g(x^i_0 \theta_0 + x^i_1 \theta_1 + \dots + x^i_n \theta_n)$$

The logistic function simply "squashes" the combined positive values to a maximum of 1, and the combined negative values to a minimum of 0. I.e. a negative weight means that a large value for this feature will actually decrease the chances of survival for a passenger. 

The input for our function $g$ is composed out of the sums of each input multiplied with its corresponding weight. Since we have made sure that our data all approximately has equal scale by using the `zscore` method, bigger values for a specific weight result in the input corresponding to that weight having more influence on the hypothesis. Similarly, negative values indicate that there is a negative correlation between the input variable and the hypothesis, while positive values indicate a positive correlation.

Below, we have used Seaborn to create a barplot that shows each of the weights and their corresponding data column names.

In [ ]:
sns.barplot(X_train.columns, theta_hat[1:])
plt.show()

This plot shows that the three most important predictive features seem to be the Passenger Class, the Fare the passenger paid in pounds and whether the passenger was male or female. We cannot directly draw conclusions about the relationship between these features and whether or not a person survived based on this plot, but we *can* use it as a starting point for further analysis.

## Analyzing the data

First, let's take a look at a basic count plot to see how many people have survived overall.

In [ ]:
sns.countplot(x='Survived',data=data)
plt.show()

We can see from this plot that approximately 350 people have survived, while approximately 550 people have died overall.

Now, lets take a look at those 3 most predictive features specifically, and see how they affect the distribution.

### 1. Gender of the passenger

In [ ]:
sns.countplot(x='Survived',hue='Sex',data=data)
plt.show()

### 2. Class of the passenger's cabin

In [ ]:
sns.countplot(x='Survived',hue='Pclass',data=data)
plt.show()

### 3. Fare the passenger paid

This is a continous variable, so we can't plot all the the distinct cases. Instead we'll use a barplot with the averages for both classes, and corresponding error bars.

In [ ]:
sns.barplot(x='Survived', y='Fare', data=data)
plt.show()

## Creating hypotheses for the data

For each of the 3 features shown above, compare their individual impact on the surival chance of the passenger. For every feature, give your own hypothesis on why this feature is likely to have a positive / negative impact on the survivability of the passenger. Also, add how this then explains the positive or negative weight the feature seems to get in the learned logistic regression model:

#### 1. Gender of the passenger

*Your answer here.*

#### 2. Class of the passenger's cabin

*Your answer here.*

#### 3. Fare the passenger paid

*Your answer here.*
